### PECOS Provider File

Although I have some suspicions about this data being bad data.

[Base Provider Enrollment File](https://data.cms.gov/Medicare-Enrollment/Base-Provider-Enrollment-File/ykfi-ffzq)

> This file is a point in time snapshot of enrollment level data for providers actively enrolled in Medicare.

https://data.cms.gov/public-provider-enrollment

> * This data was last refreshed on January 25, 2017.
> * The next data refresh is tentatively scheduled for April 2017.

> The Public Provider Enrollment data for Medicare fee-for-service includes providers who are actively approved to bill Medicare or have completed the 855O at the time the data was pulled from the Provider Enrollment and Chain Ownership System (PECOS). The release of this provider enrollment data is not related to other provider information releases such as Physician Compare or Data Transparency.

> These files are populated from PECOS and contain basic enrollment and provider information, reassignment of benefits information and practice location city, state and zip. These files are not intended to be used as real time reporting as the data changes from day to day and the files are updated only on a quarterly basis. If any information on these files needs to be updated, the provider needs to contact their respective Medicare Administrative Contractor (MAC) to have that information updated. To find your respective MAC, go to https://www.cms.gov/medicare/medicare-contracting/medicare-administrative-contractors/medicareadministrativecontractors.html.

> This data does not include information on opt-out providers. Information is redacted where necessary to protect Medicare provider privacy.

> To review more information about the Public Provider Enrollment data, please refer to the Data Dictionary and Data Guidance document.


In [3]:
import pandas
http_source = 'https://data.cms.gov/resource/7b6b-dk5v.json'
enrollments = pandas.read_json(http_source)
enrollments.head()

,enrlmt_id,first_name,gndr_sw,last_name,mdl_name,npi,pecos_asct_cntl_id,provider_type_cd,provider_type_desc,state_cd
0,I20031103000014,RHONDA,F,GRISSOM,G,1851357214,2466364161,14-30,PRACTITIONER - DIAGNOSTIC RADIOLOGY,KY
1,I20031103000013,JORGE,M,OSTOLAZA BEY,A,1831165075,5193637890,14-16,PRACTITIONER - OBSTETRICS/GYNECOLOGY,PR
2,I20031103000005,CHRISTOPHER,M,ZIEGLER,J,1003976986,7113839812,14-68,PRACTITIONER - CLINICAL PSYCHOLOGIST,PA
3,I20031103000028,DAMON,M,DELSTON,D,1720297963,7618889312,14-26,PRACTITIONER - PSYCHIATRY,NJ
4,I20031103000020,ANNA,F,MOY,NaN,1083835177,5991617698,14-41,PRACTITIONER - OPTOMETRY,NJ


In [61]:
total_enrollments = enrollments.shape[0]
print('{0} million total enrollments in PECOS dataset'.format(float(total_enrollments)/1e6))

2.055255 million total enrollments in PECOS dataset


In [62]:
group_enrollments = enrollments[pandas.notnull(enrollments['ORG_NAME'])]
total_group_enrollments = group_enrollments.shape[0]
percent_group_enrollments = round(float(total_group_enrollments)/total_enrollments*100, 2)
print('{0}% or {1} group providers'.format(percent_group_enrollments, total_group_enrollments))

20.59% or 423213 group providers


In [63]:
# number of npi's with multiple enrollments
enrollments_grouped_by_npi = enrollments.groupby('NPI')

In [65]:
group_size = enrollments_grouped_by_npi.size()
npis_with_more_than_one_enrlmnt = group_size[group_size > 1]
num_npis_with_more_than_one_enrlmnt = len(npis_with_more_than_one_enrlmnt)
num_enrollments_which_are_multiples = npis_with_more_than_one_enrlmnt.sum()
percent_enrollments_which_are_multiples = round(float(num_enrollments_which_are_multiples)/total_enrollments*100, 2)
print('{0}% of enrollments are multiples for the same NPI'.format(percent_enrollments_which_are_multiples))

14.55% of enrollments are multiples for the same NPI


In [70]:
reassignments = pandas.read_csv('pecos_data/Reassignment_Sub-_File.csv')
num_reassignments = reassignments.shape[0]
print('Number of reassigments: {0} million'.format(num_reassignments/1e6))

Number of reassigments: 1.884063 million


In [82]:
reassignments_grouped_by_enrollment_id = reassignments.groupby('REASGN_BNFT_ENRLMT_ID')
grouped_reassignments_size = reassignments_grouped_by_enrollment_id.size()

In [84]:
enrollments_with_more_than_one_reassignment = grouped_reassignments_size[grouped_reassignments_size > 1]
print('Number of enrollments with more than 1 reassignment: {0}'.format(len(enrollments_with_more_than_one_reassignment)))

Number of enrollments with more than 1 reassignment: 372163


*Herein lies an irresolvable - if a medicare beneficiary assigns benefits to more than one other we have to know the most recent assignment.*

In [79]:
#result = pd.merge(enrollments, reassignments, on='ENRLMT_ID')
reassignments.head()
enrlmnts_with_reassignments = pandas.merge(
    enrollments, reassignments, how = 'outer', left_on = 'ENRLMT_ID', right_on = 'REASGN_BNFT_ENRLMT_ID'
)
enrlmnts_with_reassignments.head()

,NPI,PECOS_ASCT_CNTL_ID,ENRLMT_ID,PROVIDER_TYPE_CD,PROVIDER_TYPE_DESC,STATE_CD,FIRST_NAME,MDL_NAME,LAST_NAME,ORG_NAME,GNDR_SW,REASGN_BNFT_ENRLMT_ID,RCV_BNFT_ENRLMT_ID
0,1003879883,8022920719,I20031103000001,14-16,PRACTITIONER - OBSTETRICS/GYNECOLOGY,PR,ANTONIO,NaN,ALVAREZ RODRIGUEZ,NaN,M,I20031103000001,O20031216000213
1,1003976986,7113839812,I20031103000005,14-68,PRACTITIONER - CLINICAL PSYCHOLOGIST,PA,CHRISTOPHER,J,ZIEGLER,NaN,M,NaN,NaN
2,1831165075,5193637890,I20031103000013,14-16,PRACTITIONER - OBSTETRICS/GYNECOLOGY,PR,JORGE,A,OSTOLAZA BEY,NaN,M,NaN,NaN
3,1851357214,2466364161,I20031103000014,14-30,PRACTITIONER - DIAGNOSTIC RADIOLOGY,KY,RHONDA,G,GRISSOM,NaN,F,I20031103000014,O20031229000649
4,1083766935,5092627794,I20031103000015,14-35,PRACTITIONER - CHIROPRACTIC,NJ,TIMOTHY,J,DIECKMAN,NaN,M,NaN,NaN
